# Tracking disease outbreaks in India using Camelot

The code here builds upon this [blog post](https://blog.socialcops.com/technology/data-science/apache-airflow-disease-outbreaks-india/). It scrapes PDFs that contain disease outbreaks data from the [IDSP website](http://idsp.nic.in/index4.php?lang=1&level=0&linkid=406&lid=3689) based on the year, from_week and to_week variables in the first cell; then extracts tables from those PDFs using Camelot.
EDIT: Currently, this notebook works only with Python 2.7, I'll update it soon to work with both Python 2 and 3.

In [1]:
import os
import re


def mkdir(path):
    """Make directory
    
    Parameters
    ----------
    path : str
    
    """
    if not os.path.exists(path):
        os.makedirs(path)


# replace vinayak with your username
data_dir = '/home/vinayak/dev/etl-camelot/data'
year = 2018
from_week = to_week = 33

In [2]:
import requests
from lxml import etree
        
        
def scrape_web(year=2018, from_week=1, to_week=52):
    """Scrape PDFs from the IDSP website
    http://idsp.nic.in/index4.php?lang=1&level=0&linkid=406&lid=3689
    
    Parameters
    ----------
    year : int
    from_week : int
    to_week : int
    
    """
    year_dir = os.path.join(data_dir, str(year))
    mkdir(year_dir)

    crawl = {}
    r = requests.get('http://idsp.nic.in/index4.php?lang=1&level=0&linkid=406&lid=3689')
    tree = etree.fromstring(r.content, etree.HTMLParser())
    table = tree.xpath('//*[@id="cmscontent"]')
    rows = table[0].cssselect('tr')
    for r in rows[1:]:
        try:
            y = int(r.cssselect('td')[0].cssselect('span')[0].xpath('text()')[0])
        except IndexError:
            y = int(r.cssselect('td')[0].cssselect('div')[0].xpath('text()')[0])
        crawl[y] = {}
        links = r.cssselect('td')[1].cssselect('a')
        for l in links:
            try:
                m = re.search(r'\d+', l.xpath('text()')[0])
            except IndexError:
                m = re.search(r'\d+', l.cssselect('span')[0].xpath('text()')[0])
            week = int(m.group(0))
            link = l.xpath('@href')[0]
            crawl[y][week] = link

    to_download = filter(lambda x: from_week <= x <= to_week, crawl[year])
    to_download = sorted(to_download)
    print 'Found {} PDF(s) for download'.format(len(to_download))
    if len(to_download):
        for w in to_download:
            print 'Downloading year {}, week {} ...'.format(year, w)
            link = crawl[year][w]
            r = requests.get(link, stream=True)
            filename = os.path.join(year_dir, '{}.pdf'.format(w))
            with open(filename, 'wb') as f:
                for chunk in r.iter_content(1024):
                    f.write(chunk)

In [3]:
scrape_web(year=2018, from_week=from_week, to_week=to_week)

Found 1 PDF(s) for download


In [4]:
import camelot


def extract_tables(year=2018, from_week=1, to_week=52):
    """Extract tables from downloaded PDFs using Camelot
    
    Parameters
    ----------
    year : int
    from_week : int
    to_week : int
    
    """
    year_dir = os.path.join(data_dir, str(year))
    if not os.path.exists(year_dir):
        print 'Year {} not found'.format(year)
        return None

    all_tables = []
    for i in range(from_week, to_week + 1):
        filename = '{}.pdf'.format(i)
        filename = os.path.join(year_dir, filename)
        print 'Processing {} ...'.format(filename)
        tables = camelot.read_pdf(filename, pages='3-end', line_size_scaling=40)
        print 'Found {} tables(s)'.format(tables.n)
        all_tables.append(tables)
        
    return all_tables

In [5]:
all_tables = extract_tables(year=2018, from_week=from_week, to_week=to_week)

Processing /home/vinayak/dev/etl-camelot/data/2018/33.pdf ...
Found 7 tables(s)


In [6]:
import pandas as pd


headers = [
    'unique_id',
    'state',
    'district',
    'disease_illness',
    'num_cases',
    'num_deaths',
    'date_of_start_of_outbreak',
    'date_of_reporting',
    'current_status',
    'comment_action_taken',
    'reported_late',
    'under_surveillance'
]
ten_headers = [
    'unique_id',
    'state',
    'district',
    'disease_illness',
    'num_cases',
    'num_deaths',
    'date_of_start_of_outbreak',
    'date_of_reporting',
    'current_status',
    'comment_action_taken'
]
nine_headers = [
    'unique_id',
    'state',
    'district',
    'disease_illness',
    'num_cases',
    'num_deaths',
    'date_of_start_of_outbreak',
    'current_status',
    'comment_action_taken'
]


def append_tables(all_tables):
    """Append all tables in PDFs
    
    Parameters
    ----------
    all_tables : list
    
    """
    df = pd.DataFrame(columns=headers)
    for tables in all_tables:
        for table in tables:
            columns = list(table.df.iloc[0])
            if table.shape[1] == 10:
                temp = table.df.copy()
                if 'unique' in columns[0].lower():
                    temp = temp.iloc[1:]
                temp.columns = ten_headers
                temp['reported_late'] = False
                temp['under_surveillance'] = False
                df = pd.concat([df, temp], sort=False)
            elif table.shape[1] == 9:
                temp = table.df.copy()
                if 'disease' in columns[0].lower():
                    c = temp.iloc[0]
                    temp = temp.iloc[2:]
                    temp.columns = nine_headers
                    if 'reportedlate' in c[0].lower().replace(' ', ''):    
                        temp['reported_late'] = True
                        temp['under_surveillance'] = False
                    elif 'undersurv' in c[0].lower().replace(' ', ''):
                        temp['reported_late'] = False
                        temp['under_surveillance'] = True
                    df = pd.concat([df, temp], sort=False)
                else:
                    temp.columns = nine_headers
                    temp['reported_late'] = True
                    temp['under_surveillance'] = False
                    df = pd.concat([df, temp], sort=False)
    return df

The append logic above is just a heuristic, which would need to change if the table structure changes in future.

In [7]:
df = append_tables(all_tables)
df

,unique_id,state,district,disease_illness,num_cases,num_deaths,date_of_start_of_outbreak,date_of_reporting,current_status,comment_action_taken,reported_late,under_surveillance
1,AR/NMI/2018/33/1159,Arunachal Pradesh,Namsai,Japanese Encephalitis,01,00,16-08-18,20-18-18,Under Surveillance,Cases of fever with encephalitis reported from...,False,False
2,JH/RCH/2018/33/1160,Jharkhand,Ranchi,Food Poisoning,10,00,13-08-18,13-08-18,Under Control,Cases of vomiting with pain abdomen reported f...,False,False
3,KL/KOZ/2018/33/1161,Kerala,Kozhikode,Viral Hepatitis A,26,00,16-08-18,20-08-18,Under Surveillance,Cases of jaundice with fever reported fro...,False,False
4,PB/PTL/2018/33/1162,Punjab,Patiala,Acute Diarrheal Diseases,71,00,14-08-18,18-08-18,Under Surveillance,Cases of loose stools with vomiting repo...,False,False
0,PB/MUK/2018/33/1163,Punjab,Muktsar,Viral Hepatitis E,30,00,17-08-18,23-08-18,Under Surveillance,Cases of jaundice reported from Gidderbah...,False,False
1,RJ/DGR/2018/33/1164,Rajasthan,Dungarpur,Acute Diarrheal Diseases,10,01,21-08-18,21-08-18,Under Control,Cases of loose stools with vomiting repo...,False,False
2,TN/TNR/2018/34/1165,Tamil Nadu,Tiruvannamalai,Viral Hepatitis A,11,00,16-08-18,20-08-18,Under Control,Cases of jaundice reported from Village Kolun...,False,False
0,TN/ERD/2018/34/1166,Tamil Nadu,Erode,Food Borne Illness,588,00,15-08-18,16-08-18,Under Surveillance,Cases of vomiting with loose stools repo...,False,False
1,TL/KRN/2018/33/1167,Telangana,Krimnagar,Dengue,06,00,13-08-18,21-08-18,Under Control,Cases of fever reported from Village Patarlapa...,False,False
0,WB/MDW/2018/33/1168,West Bengal,Paschim Medinipur,Dengue,497,00,06-08-18,06-08-18,Under Surveillance,Cases of fever with rash reported from Village...,False,False


Now that you have a clean table, you can do further analysis, or just load it into a data warehouse for querying/visualization.

In [8]:
filename = os.path.join(data_dir, 'data.csv')
df.to_csv(filename, index=False, quoting=1, encoding='utf-8')